In [1]:
%pip install azure-iot-hub
%pip install azure-iot-device

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import sys
import os
from time import sleep
from azure.iot.hub import IoTHubRegistryManager
from azure.iot.hub.models import Twin, TwinProperties, QuerySpecification, QueryResult
from azure.iot.device import IoTHubDeviceClient, Message

IOTHUB_CONNECTION_STRING = os.getenv('AZURE_IOT_HUB_CONNECTION_STRING')
DEVICE_ID = "device-1"

AZURE_DEVICE_1_CONNECTION_STRING = os.getenv('AZURE_DEVICE_1_CONNECTION_STRING')

In [12]:

import time
# Create instance of the device client
d = IoTHubDeviceClient.create_from_connection_string(AZURE_DEVICE_1_CONNECTION_STRING)

# Connect to the IoT hub
d.connect()

# Number of messages to send
NUM_MESSAGES = 100

# Start the timer
start_time = time.time()

# Send the messages in a loop
for i in range(NUM_MESSAGES):
    '''
    # Property name and value
    property_name = "glucose"
    property_value = 267
    # Report the new property value
    d.patch_twin_reported_properties({property_name: property_value})
    '''
    
    msg = Message(f"Hi {i}")
    d.send_message(msg)

# Stop the timer
end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time

# Disconnect from the IoT hub
d.disconnect()

# Print the result
print(f"Time taken to send {NUM_MESSAGES} messages: {time_taken} seconds")


Time taken to send 100 messages: 5.976132392883301 seconds


In [13]:
12*24


288

In [4]:

def iothub_service_sample_run(result):
    try:
        iothub_registry_manager = IoTHubRegistryManager(IOTHUB_CONNECTION_STRING)

        new_tags = {
                'patient' : {
                    'region' : 'US',
                    'id' : 'pt-34322'
                }
            }

        twin = iothub_registry_manager.get_twin(DEVICE_ID)
        twin_patch = Twin(tags=new_tags, properties= TwinProperties(desired={'glucose' : 120}, reported={'glucose' : 250}))
        twin = iothub_registry_manager.update_twin(DEVICE_ID, twin_patch, twin.etag)

        # Add a delay to account for any latency before executing the query
        sleep(1)

        query_spec = QuerySpecification(query="SELECT * FROM devices WHERE tags.patient.id = 'pt-34322'")
        query_result = iothub_registry_manager.query_iot_hub(query_spec, None, 100)
        result = query_result
        print("Devices: {}".format(', '.join([twin.device_id for twin in query_result.items])))

        print()

        query_spec = QuerySpecification(query="SELECT * FROM devices WHERE tags.patient.id = 'pt-34322' AND properties.desired.glucose = 120")
        query_result = iothub_registry_manager.query_iot_hub(query_spec, None, 100)
        print("Devices: {}".format(', '.join([twin.device_id for twin in query_result.items])))

    except Exception as ex:
        print("Unexpected error {0}".format(ex))
        return
    except KeyboardInterrupt:
        print("IoT Hub Device Twin service sample stopped")
        
    return result

In [5]:
if __name__ == '__main__':
    print("Starting the Python IoT Hub Device Twin service sample...")
    print()

    iothub_service_sample_run()

Starting the Python IoT Hub Device Twin service sample...



TypeError: iothub_service_sample_run() missing 1 required positional argument: 'result'

In [ ]:
result = None
result = iothub_service_sample_run(result)
for twin in result.items:
    print (twin.properties.reported.get('glucose'))

Devices: device-1

Devices: device-1
267
